In [43]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
#Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [44]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

loader = WebBaseLoader("https://python.langchain.com/docs/concepts/output_parsers/")
loaded = loader.load()
splitting = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap=50)
splitted_result = splitting.split_documents(loaded)
splitted_result
embedding = OpenAIEmbeddings()



db = FAISS.from_documents(splitted_result,embedding)
query = "LangChain has lots of different types of output parsers."
result = db.similarity_search(query)
print(result[0].page_content)

Useful when you are using LLMs to generate structured data, or to normalize output from chat models and LLMs.
LangChain has lots of different types of output parsers. This is a list of output parsers LangChain supports. The table below has various pieces of information:


In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context
<context>
{context}
</context>
question: {input}
    """
)
document_chain = create_stuff_documents_chain(llm,prompt)

In [46]:
retriver = db.as_retriever()


In [47]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retrieval = RunnableParallel(
    {"context": retriver, "input": RunnablePassthrough()}
)

In [48]:
# from langchain.chains import create_retrieval_chain

# retrivel_chain = create_retrieval_chain(retriver,document_chain)
# response = retrivel_chain.invoke({"input": "is output parser can  be used for reasoning?"})
# response['answer']
from langchain_core.output_parsers import StrOutputParser
op_parse = StrOutputParser()
chains = setup_and_retrieval |prompt|llm| op_parse
response = chains.invoke("is output parser can  be used for reasoning?")
response

'The provided context does not explicitly mention whether output parsers can be used for reasoning. It primarily focuses on the role of output parsers in transforming model outputs into structured representations and suggests using function/tool calling instead of output parsing. Therefore, based on the provided information, it is unclear if output parsers are intended for reasoning.'

In [49]:
async for chunk in chains.astream_events(
    "is output parser can  be used for reasoning?", version="v1", 
):
    print("-" * 40)
    print(chunk)

----------------------------------------
{'event': 'on_chain_start', 'run_id': '110f3b23-77ad-42d0-8724-f3c49c2d7c3a', 'name': 'RunnableSequence', 'tags': [], 'metadata': {}, 'data': {'input': 'is output parser can  be used for reasoning?'}, 'parent_ids': []}
----------------------------------------
{'event': 'on_chain_start', 'name': 'RunnableParallel<context,input>', 'run_id': '541f7ab8-f97a-41e5-865c-de41a720c408', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {}, 'parent_ids': []}
----------------------------------------
{'event': 'on_retriever_start', 'name': 'VectorStoreRetriever', 'run_id': '09f1cce5-138d-461d-b60c-0fb55958937d', 'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'OpenAIEmbeddings'}, 'data': {'input': {'query': 'is output parser can  be used for reasoning?'}}, 'parent_ids': []}
----------------------------------------
{'event': 'on_chain_start'